<a href="https://colab.research.google.com/github/sathuvivek/nlp-in-practice/blob/master/usingScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import files
uploaded = files.upload()


Saving ScriptIncludesInfo.csv to ScriptIncludesInfo (2).csv


In [0]:
import io
import pandas as pd
df_idf = pd.read_csv(io.BytesIO(uploaded['ScriptIncludesInfo.csv']), encoding='cp1252')

In [16]:
print("name:\n\n",df_idf.name)
print("description:\n\n",df_idf.description)
#print("script:\n\n",df_idf.script)




name:

 0                                   NaN
1         AbstractAjaxDataUpdateChecker
2           AbstractAjaxListDragChecker
3                 AbstractAjaxProcessor
4                      AbstractDBObject
5                   AbstractDeclaration
6                          AbstractList
7        AbstractOrignatorFinderService
8                     AbstractParameter
9                    AbstractReconciler
10      AbstractSchedulePageRESTWrapper
11              AbstractScriptProcessor
12         AbstractTimelineSchedulePage
13                  AbstractTransaction
14           AbstractUpdateUIActionUtil
15                    AbstractValidator
16                   ACamTableProcessor
17                         AccessFinder
18                         AccessHelper
19                           AccessUtil
20                     AccessUtilErrors
21                 ACiscoCamTableSensor
22                   AclBasedChoiceList
23                         ACLDescriber
24                ACL_Access_Aut

In [33]:
import re
def pre_process(text):
    
    # lowercase
   # text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf['text'] = df_idf['name'] + df_idf['script'].fillna('')
df_idf['text'] = df_idf['text'].astype(str)
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the first 'text'
df_idf['text']

0                                                     nan
1       AbstractAjaxDataUpdateCheckervar AbstractAjaxD...
2       AbstractAjaxListDragCheckervar AbstractAjaxLis...
3       AbstractAjaxProcessor Base ajax processor clas...
4       AbstractDBObject Discovery class Provides comm...
5       AbstractDeclarationvar AbstractDeclaration Cla...
6       AbstractListgs include PrototypeServer var Abs...
7       AbstractOrignatorFinderService Abstract class ...
8       AbstractParametervar AbstractParameter Class c...
9       AbstractReconciler Discovery gs include Protot...
10      AbstractSchedulePageRESTWrappervar AbstractSch...
11      AbstractScriptProcessorvar AbstractScriptProce...
12      AbstractTimelineSchedulePage Class Imports var...
13      AbstractTransactiongs include PrototypeServer ...
14      AbstractUpdateUIActionUtilvar AbstractUpdateUI...
15      AbstractValidator Abstract class for preparing...
16      ACamTableProcessorvar ACamTableProcessor Class...
17            

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("stopwords.txt")

#get the text column 
docs=df_idf['text'].fillna(' ').tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['glideaggregate', 'gliderecord'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [35]:
word_count_vector.shape


(6999, 152378)

In [36]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [37]:
tfidf_transformer.idf_


array([8.06190596, 8.46737107, 8.75505314, ..., 9.16051825, 9.16051825,
       9.16051825])

In [0]:
df_test=pd.read_csv(io.BytesIO(uploaded['ScriptIncludesInfo.csv']), encoding='cp1252')
df_test['text'] = df_test['name'] + df_test['script'].fillna('')
df_test['text'] = df_test['text'].astype(str)
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_name=df_test['name'].tolist()
docs_script=df_test['script'].tolist()

In [0]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [40]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[1]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Name=====")
print(docs_title[1])
print("\n=====script=====")
print(docs_script[1])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Name=====
AbstractAjaxDataUpdateChecker

=====script=====
var AbstractAjaxDataUpdateChecker = Class.create();
AbstractAjaxDataUpdateChecker.prototype = Object.extendsObject(AbstractAjaxProcessor, {
   updateData: function() {
      var itemID = this.getParameter('sysparm_item_id');
      var userLanguage = this.getParameter('sysparm_user_language');
      var table = this.getTableName();
      var result = "";
      if (itemID != "" && table != null){
         var gr = new GlideRecord(table);
         gr.addQuery('sys_id', itemID);
         gr.query();
         if (gr.next()){
            var values = this.getNewValues(itemID, gr.state);
            if (values){
               gr.state = values[0];
               gr.blocked = values[1];
               gr.blocked_reason = values[2];
               gr.update();
               result = values.join("|");
            }
         }
      }
      return result;
   },

   whatWillChange: function() {
      var itemID = this.getParameter('